# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [11]:
%pip install --upgrade --quiet langchain langchain-community langchain_openai

Note: you may need to restart the kernel to use updated packages.


If you'd like to use LangSmith, uncomment the below:

In [12]:
import getpass
import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [13]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")


#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows, macOS, and Linux.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  macOS

1. **Install via Homebrew**  
   Open the Terminal and run:
   ```bash
   brew install ollama
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Linux

1. **Install Ollama**  
   Open a terminal and run:
   ```bash
   curl -fsSL https://ollama.com/install.sh | sh
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [14]:
from langchain_community.llms import Ollama

# Connect to Ollama on Windows host from container
#model = Ollama(model="phi3", base_url="http://host.docker.internal:11434")

In [15]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [16]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [17]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [18]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [19]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

```json
{
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}
```


System: You are an advanced assistant with access to a variety of tools. Here is what each tool can do and its description:

multiply(base_number: float, exponent: int) -> complex - Calculate the power of a number by raising it to a given integer (real numbers only). Returns result in standard form for real-world application clarity. Includes handling edge cases like 0 raised to any positive or negative power and infinities as necessary outputs.
add(first_number: int, second_number: int) -> complex - Add two integers together using precise arithmetic representation (complex numbers format). Ensures that the result is given in standard form for real-world application clarity when dealing with large sums where precision matters significantly.
log(value: float) -> str - Calculate and return a string describing the natural logarithm of an input number, including handling special cases such as inputs less than

## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [20]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [21]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [22]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [23]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [24]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

## Weather based tool

In [25]:
import random
from langchain_core.tools import tool

@tool
def generate_weather(day: str) -> str:
    """Generate random weather for a given day of the week."""
    weather_conditions = ["sunny", "rainy", "cloudy"]
    day_cleaned = day.lower().strip()
    
    # Add some logic to make it realistic
    if day_cleaned in ["saturday", "sunday"]:
        # Weekends are slightly more likely to be nice weather :)
        weather = random.choices(
            weather_conditions, 
            weights=[40, 20, 40]  # sunny more likely
        )[0]
    else:
        weather = random.choice(weather_conditions)
    
    return f"Weather for {day}: {weather}"

@tool
def suggest_clothing(weather: str) -> str:
    """Suggest clothing based on weather condition."""
    weather_clean = weather.lower()
    
    if "sunny" in weather_clean:
        return "Perfect for shorts, dress, or light clothing"
    elif "rainy" in weather_clean:
        return "Wear pants, long sleeves, and a waterproof jacket"
    elif "cloudy" in weather_clean:
        return "Wear layers - pants and a light sweater"
    else:
        return "Wear comfortable clothing"
    
@tool
def suggest_accessories(weather: str) -> str:
    """Suggest accessories and items to bring based on weather."""
    weather_clean = weather.lower()
    
    if "sunny" in weather_clean:
        return "Bring a water bottle, sunglasses, and sunscreen"
    elif "rainy" in weather_clean:
        return "Bring an umbrella and a waterproof bag"
    elif "cloudy" in weather_clean:
        return "Bring a light jacket, just in case"
    else:
        return "No special accessories needed"

# Your tools list
tools = [generate_weather, suggest_clothing, suggest_accessories]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
generate_weather
Generate random weather for a given day of the week.
{'day': {'title': 'Day', 'type': 'string'}}
--
suggest_clothing
Suggest clothing based on weather condition.
{'weather': {'title': 'Weather', 'type': 'string'}}
--
suggest_accessories
Suggest accessories and items to bring based on weather.
{'weather': {'title': 'Weather', 'type': 'string'}}


In [26]:
generate_weather.invoke({"day": "Saturday"})

'Weather for Saturday: cloudy'

In [27]:
suggest_clothing.invoke({"weather": "sunny"})

'Perfect for shorts, dress, or light clothing'

In [28]:
suggest_accessories.invoke({"weather": "rainy"})

'Bring an umbrella and a waterproof bag'

## Creating New Prompts

In [29]:
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

generate_weather(day: str) -> str - Generate random weather for a given day of the week.
suggest_clothing(weather: str) -> str - Suggest clothing based on weather condition.
suggest_accessories(weather: str) -> str - Suggest accessories and items to bring based on weather.


* Creating new prompt

In [30]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

* Testing New Prompt

In [31]:
chain = prompt | model
message = chain.invoke({"input": "What will the weather be like on Saturday?"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

```json
{
  "name": "generate_weather",
  "arguments": {
    "day": "Saturday"
  }
}
```


* JSONOutputParser

In [32]:
chain = prompt | model | JsonOutputParser()
message = chain.invoke({"input": "What will the weather be like on Saturday?"})
print(message)  # Should be clean dict: {'name': 'generate_weather', 'arguments': {'day': 'Saturday'}}

{'name': 'generate_weather', 'arguments': {'day': 'Saturday'}}


* Invoking the tool

In [33]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

In [34]:
invoke_tool({"name": "generate_weather", "arguments": {"day": "Saturday"}})

'Weather for Saturday: cloudy'

In [35]:
invoke_tool({"name": "suggest_clothing", "arguments": {"weather": "sunny"}})

'Perfect for shorts, dress, or light clothing'

In [36]:
chain.invoke({"input": "What should I wear on a cloudy day?"})

{'name': 'suggest_clothing', 'arguments': {'weather': 'cloudy'}}

In [38]:
# Test accessories
chain.invoke({"input": "What do I need to bring if it's rainy?"})

{'name': 'generate_weather', 'arguments': {'day': ''}}

In [39]:
# Test different days
chain.invoke({"input": "What will the weather be like on Monday?"})

{'name': 'generate_weather', 'arguments': {'day': 'Monday'}}

* Lets Put it together

In [40]:
chain = prompt | model | JsonOutputParser() | invoke_tool
result = chain.invoke({"input": "What will the weather be like on Saturday?"})
print(result)  # Should output something like: "Weather for Saturday: sunny"

Weather for Saturday: sunny


* Add Conversation Wrapper

In [41]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

def enhanced_invoke_tool(data):
    """Invoke tool while preserving original input."""
    tool_result = invoke_tool(data["parsed_request"])
    return {
        "tool_result": tool_result,
        "original_input": data["input"]
    }

def make_conversational(data):
    """Make tool results more conversational using LLM."""
    conversation_prompt = f"""
    User asked: "{data['original_input']}"
    Tool result: "{data['tool_result']}"
    
    Rewrite this as a friendly, conversational response that directly answers the user's question.
    Make it natural and helpful, like you're talking to a friend but keep it to 1sentence.
    """

    # Use your LLM to make it conversational
    friendly_chain = ChatPromptTemplate.from_template(conversation_prompt) | model
    return friendly_chain.invoke({})

In [42]:
# Enhanced chain that preserves original input
chain = (
    RunnablePassthrough.assign(parsed_request=prompt | model | JsonOutputParser())
    | enhanced_invoke_tool
    | make_conversational
)

In [43]:
# Test it!
result = chain.invoke({"input": "What will the weather be like on Saturday?"})
print(result)

Hey! Looks like it will be cloudy on Saturday—perhaps best to have an umbrella just in case there's some rain? Hope your plans go well despite the weather forecast!


## Observations:
* it was so interesting how these decorators works. 
* it automatically converts any Python function into something an LLM can understand and call through JSON
* This pattern of using AI to improve AI outputs seems like a powerful technique I'll definitely use in future projects.